# Assignment 6

### Part A

In [72]:
import numpy as np
import pandas as pd

Consider a DataFrame giving information about the price of a stock in 1 year, and the probability of that price occurring.

In [73]:
data = pd.DataFrame(data = {"Price in 1 Year" : [32, 43, 54],
                            "Probability" : [0.3, 0.4, 0.3]})
                    
data

Price in 1 Year  Probability
0               32          0.3
1               43          0.4
2               54          0.3

Write the function ```exp_return```, which calculates the expected return of the stock, given the current stock price ```P```.

In [74]:
def exp_return(data, P):
    # Write your code here
    # exp_return(data, 40) should output 0.07500000000000002
    expected_price = (data['Price in 1 Year'] * data['Probability']).sum()
    expected_return = (expected_price - P) / P 

    return expected_return

Write the function ```volatility```, which calculates the expected return of the stock, given the current stock price ```P```.

In [75]:
def volatility(data, P):
    # Write your code here
    # volatility(data,40) should output 0.21301408404140795
    # Calculate the expected price
    expected_price = (data['Price in 1 Year'] * data['Probability']).sum()
    
    # Calculate the expected return
    expected_return = (expected_price - P) / P
    
    # Calculate the variance of returns
    data['variance'] = (data['Probability'] * ((data['Price in 1 Year'] - P)/P - expected_return )**2)
    variance = data['variance'].sum()
    sd = np.sqrt(variance)
    return sd


Write the function ```interval```, which calculates the 95% confidence interval for the expected return of the data. The corresponding Z score is 1.96. The function should return a list of size 2, with the first element representing the lower bound, and the second element representing the upper bound. 

In [76]:
def interval(data, P):
    # Write your code here
    # interval(data, 40) should output [-0.16604812797447732, 0.3160481279744774]

    expected_returns = [(price - P) / P for price in data['Price in 1 Year']]
    mean_return = sum(prob * ret for prob, ret in zip(data['Probability'], expected_returns))
    sd = np.sqrt(sum(prob * (ret - mean_return) ** 2 for prob, ret in zip(data['Probability'], expected_returns)))
    sem = sd / np.sqrt(len(data['Price in 1 Year']))

    z_score = 1.96
    lower = mean_return - z_score * sem
    upper = mean_return + z_score * sem
    
    return [lower, upper]

Write the function ```exp_return_capm```, which, given the beta of a security ```beta```, the return on the market ```market_return```,  and the risk free interest rate ```rf```, calculates the expected return of the security.

In [77]:
def exp_return_capm(beta, market_return, rf):
    # Write your code here
    # exp_return_capm(-0.6,0.1, 0.02) should output -0.028
    return rf + beta * (market_return - rf)

### Part B

Write the function ```clean_data```, which will take a path to a csv file like the one provided. This function should remove all observations with a non number ```RET```, and remove all securities with (strictly) less than 500 observations. Your returned DataFrame should have 4 (non-index) columns, `PERMNO`, `date`, `CUSIP`, and `RET`.

In [78]:
def clean_data(path):
    # This should yield data with 403316 rows × 4 columns
    df = pd.read_csv(path)    
    df['RET'] = pd.to_numeric(df['RET'], errors='coerce')
    df = df.dropna(subset=['RET'])
    counts = df['PERMNO'].value_counts()
    df = df[df['PERMNO'].isin(counts[counts >= 500].index)]
    return df

Write the function ```avg_return```, which will calculate the average return for each security. This function should take in a DataFrame ```data```, formatted like one outputted by the function ```clean_data```. The results should be returned in a DataFrame formatted like the one supplied.

Additionally, write the function ```security_volatilities```, which does the same thing as ```avg_return```, except it returns the volatilities of the returns of the securities.

In [79]:
def avg_return(data):
    # This should yield data with 679 rows × 2 columns
    avg_returns = data.groupby('CUSIP')['RET'].mean().reset_index()
    avg_returns.columns = ['CUSIP', 'Average Return']
    
    return avg_returns


In [80]:
def security_volatilities(data):
    # Write your code here
    # This should yield data with 679 rows × 2 columns

    volatilities = data.groupby('CUSIP')['RET'].std().reset_index()
    volatilities.columns = ['CUSIP', 'Volatility']
    
    return volatilities

Write the function ```correlation```, which will return the correlation between average return and volatility. This function takes two DataFrames, ```ret_data``` and ```vol_data```, which are the outputs of ```avg_return``` and ```security_volatilities``` respectively.

In [81]:
def correlation(ret_data, vol_data):
    # Write your code here
    # correlation(rets, vols) should yield 0.44837090727059614
    merged_data = pd.merge(ret_data, vol_data, on='CUSIP')
    correlation_matrix = merged_data[['Average Return', 'Volatility']].corr()
    return correlation_matrix.loc['Average Return', 'Volatility']
